# Submitting form data to NACC Flywheel API



In [ ]:
from getpass import getpass
import logging
import os

from flywheel import Client, FileSpec

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

## Flywheel Access

This script requires the API KEY for a user with upload capability for this operation.

To get the API key, login as the user to the NACC Flywheel instance.
Find the "avatar" in the upper right corner (generally a circle with your initials).
Click the avatar dropdown, and select "Profile".
Under "Flywheel Access" at the bottom of the resulting page, click "Generate API Key".

Choose a key name relevant to upload, set the expiration date, and create the API Key.
Copy the API Key since you wont be able to access the value later.
This key should be kept secret.

### Load secret key in script

This script is for the purposes of demonstration, so we just prompt you for the key.

For your own scripting please use appropriate secret management for your execution environment.

In [ ]:
API_KEY = getpass('Enter API_KEY here: ')

## Connect to Flywheel

The API Key is used to connected to Flywheel.
To do this with the Python SDK, we create an SDK client that we will use.

In [ ]:
fw = Client(API_KEY if 'API_KEY' in locals() else os.environ.get('FW_KEY'))

log.info('You are now logged in as %s to %s', fw.get_current_user()['email'], fw.get_config()['site']['api_url'])

## Identify project for upload

Each center is associated with a Flywheel group, and each data type has an "ingest" Flywheel project for submission.

For data files that have data for many participants, such as forms, the data is submitted as a CSV where each line is a data record for a participant.
These files are attached to the ingest project as shown here.

> Data files that have a one to one relationship with participant, such as images, are uploaded differently.

### Indicating the group for the center

The group label for a center can be found by logging into Flywheel

In [ ]:
group_label = 'example-adrc'

Data is uploaded to a Flywheel project within this group with a prefix `ingest-`.

Form data should be submitted using the following settings

In [ ]:
ingest_label = 'ingest-form'
file_type = 'text/csv'

Once the group and ingest project are identified, the SDK client is used to lookup the project.

In [ ]:
ingest_project = fw.lookup(f"{group_label}/{ingest_label}")

Your script should have error handling to ensure the project was found.

# Get file to upload

In [ ]:

contents = ""
file_size = 0
file_spec = FileSpec("filename", contents="", content_type=file_type, size=file_size)

# Upload file

In [ ]:


if ingest_project:
    ingest_project.upload_file(file_spec)